In [ ]:
# | default_exp mkdocs

In [ ]:
# | export

from typing import *

import os
import re
import collections
from pathlib import Path
import textwrap
import shutil
import types
import pkgutil
import importlib
import subprocess # nosec: B404

import typer

from configupdater import ConfigUpdater, Section
from configupdater.option import Option

from configparser import ConfigParser

from nbdev_mkdocs.package_data import get_root_data_path

In [ ]:
import pytest
import numpy as np
from tempfile import TemporaryDirectory
import yaml

## Create new

### Add requirements to settings

In [ ]:
# | export


def _add_requirements_to_settings(root_path: str):
    """Adds requirments needed for mkdocs to settings.ini

    Params:
        root_path: path to where the settings.ini file is located

    """
    _requirements_path = get_root_data_path() / "requirements.txt"
    with open(_requirements_path, "r") as f:
        _new_req_to_add = f.read()
        lines = _new_req_to_add.split("\n")
        lines = [s.strip() for s in lines]
        lines = [s for s in lines if s != ""]
        _new_req_to_add = " \\\n".join(lines)

    setting_path = Path(root_path) / "settings.ini"
    if not setting_path.exists():
        typer.secho(
            f"Path '{setting_path.resolve()}' does not exists! Please use --root_path option to set path to setting.ini file.",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=1)

    try:

        updater = ConfigUpdater()
        updater.read(setting_path)
    except Exception as e:
        typer.secho(
            f"Error while reading '{setting_path.resolve()}': {e}",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=2)

    try:
        if "requirements" not in updater["DEFAULT"]:
            updater["DEFAULT"]["requirements"] = ""

        old_req: str = updater["DEFAULT"]["requirements"].value  # type: ignore

        def remove_leading_spaces(s: str) -> str:
            return "\n".join([x.lstrip() for x in s.split("\n")])

        old_req = remove_leading_spaces(old_req)
        new_req = remove_leading_spaces(_new_req_to_add)
        if new_req in old_req:
            typer.secho(f"Requirements already added to '{setting_path.resolve()}'.")
            return

        req = old_req + " \\\n" + new_req
        req = textwrap.indent(req, " " * 4)

        req_option = Option(key="requirements", value=req)
        updater["DEFAULT"]["requirements"] = req_option
    except Exception as e:
        typer.secho(
            f"Error while updating requiremets in '{setting_path.resolve()}': {e}",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=3)

    updater.update_file()

    typer.secho(f"Requirements added to '{setting_path.resolve()}'.")

    return

In [ ]:
with TemporaryDirectory() as d:
    shutil.copyfile(Path("..") / "settings.ini", Path(d) / "settings.ini")

    updater = ConfigUpdater()
    updater.read(Path(d) / "settings.ini")
    updater["DEFAULT"]["requirements"] = Option(
        key="requirements", value="\\\n  nbdev>=2.3.7 \\\n  typer[all]==0.6.1"
    )
    updater.update_file()

    assert "mkdocs" not in updater["DEFAULT"]["requirements"].value

    # testing adding requirements
    _add_requirements_to_settings(d)

    updater = ConfigUpdater()
    updater.read(Path(d) / "settings.ini")
    founded = re.findall("mkdocs[\w_\-\[\]]*", updater["DEFAULT"]["requirements"].value)
    assert len(founded) == 5, founded

    # do nothin if the requirements are already added
    _add_requirements_to_settings(d)

    updater = ConfigUpdater()
    updater.read(Path(d) / "settings.ini")
    founded = re.findall("mkdocs[\w_\-\[\]]*", updater["DEFAULT"]["requirements"].value)
    assert len(founded) == 5, founded

    print(updater)

Requirements added to '/tmp/tmphw_tdye6/settings.ini'.
Requirements already added to '/tmp/tmphw_tdye6/settings.ini'.
[DEFAULT]
# All sections below are required unless otherwise specified.
# See https://github.com/fastai/nbdev/blob/master/settings.ini for examples.

### Python library ###
repo = nbdev_mkdocs
lib_name = %(repo)s
version = 0.0.1
min_python = 3.7
license = apache2

### nbdev ###
doc_path = _docs
lib_path = nbdev_mkdocs
nbs_path = nbs
recursive = True
tst_flags = notest
put_version_in_init = True
black_formatting = True

### Docs ###
branch = main
custom_sidebar = False
doc_host = https://%(user)s.github.io
doc_baseurl = /%(repo)s
git_url = https://github.com/%(user)s/%(repo)s
title = %(lib_name)s

### PyPI ###
audience = Developers
author = airt
author_email = info@airt.ai
copyright = 2022 onwards, %(author)s
description = Extension to nbdev for usage of Material for Mkdocs instead of Quarto
keywords = nbdev jupyter notebook python
language = English
status = 3
user = ai

### Create mkdocs dir

In [ ]:
# | export


def _create_mkdocs_dir(root_path: str):
    mkdocs_template_path = get_root_data_path() / "mkdocs_template"
    if not mkdocs_template_path.exists():
        typer.secho(
            f"Unexpected error: path {mkdocs_template_path.resolve()} does not exists!",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=4)
    dst_path = Path(root_path) / "mkdocs"
    if dst_path.exists():
        typer.secho(
            f"Directory {dst_path.resolve()} already exist, skipping its creation.",
        )
    else:
        shutil.copytree(mkdocs_template_path, dst_path)
        #         shutil.move(dst_path.parent / "mkdocs_template", dst_path)
        typer.secho(
            f"Directory {dst_path.resolve()} created.",
        )

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    _create_mkdocs_dir(d)

    print("\n".join([str(p) for p in (Path(d) / "mkdocs").glob("**/*")]))

Directory /tmp/tmpkfu665sd/mkdocs created.
/tmp/tmpkfu665sd/mkdocs/overrides
/tmp/tmpkfu665sd/mkdocs/site
/tmp/tmpkfu665sd/mkdocs/overrides/main.html
/tmp/tmpkfu665sd/mkdocs/site/stylesheets
/tmp/tmpkfu665sd/mkdocs/site/images
/tmp/tmpkfu665sd/mkdocs/site/javascripts
/tmp/tmpkfu665sd/mkdocs/site/assets
/tmp/tmpkfu665sd/mkdocs/site/404.html
/tmp/tmpkfu665sd/mkdocs/site/stylesheets/extra.css
/tmp/tmpkfu665sd/mkdocs/site/images/favicon.ico
/tmp/tmpkfu665sd/mkdocs/site/javascripts/extra.js
/tmp/tmpkfu665sd/mkdocs/site/assets/stylesheets
/tmp/tmpkfu665sd/mkdocs/site/assets/images
/tmp/tmpkfu665sd/mkdocs/site/assets/javascripts
/tmp/tmpkfu665sd/mkdocs/site/assets/_mkdocstrings.css
/tmp/tmpkfu665sd/mkdocs/site/assets/stylesheets/main.3de6f41f.min.css
/tmp/tmpkfu665sd/mkdocs/site/assets/stylesheets/palette.cc9b2e1e.min.css.map
/tmp/tmpkfu665sd/mkdocs/site/assets/stylesheets/main.3de6f41f.min.css.map
/tmp/tmpkfu665sd/mkdocs/site/assets/stylesheets/palette.cc9b2e1e.min.css
/tmp/tmpkfu665sd/mkdoc

### Create Mkdocs.yml

In [ ]:
# | export

_mkdocs_template_path = get_root_data_path() / "mkdocs_template.yml"

In [ ]:
assert _mkdocs_template_path.exists()

In [ ]:
# | export

with open(_mkdocs_template_path, "r") as f:
    _mkdocs_template = f.read()

In [ ]:
print(_mkdocs_template)

# Site
site_name: {title}
site_url: {doc_host}{doc_baseurl}
site_author: {author}
site_description: {description}
  
# Repository
repo_name: {repo}
repo_url: {git_url}
edit_uri: ""

copyright: {copyright}

docs_dir: docs
site_dir: site

plugins:
- literate-nav:
    nav_file: SUMMARY.md
- search
- mkdocstrings:
    handlers:
      python:
        import:
            - https://docs.python.org/3/objects.inv
        options:
            heading_level: 2
            show_category_heading: true
            show_root_heading: true
            show_root_toc_entry: true
            show_signature_annotations: true
            show_if_no_docstring: true
            
markdown_extensions:
    - pymdownx.arithmatex:
        generic: true
    - pymdownx.inlinehilite
    - pymdownx.details
    - pymdownx.emoji
    - pymdownx.magiclink
    - pymdownx.superfences
    - pymdownx.tasklist
    - pymdownx.highlight:
        linenums: false
    - pymdownx.snippets:
        check_paths: true
    - pymdownx.t

In [ ]:
# | export
def _get_kwargs_from_settings(
    settings_path: Path, mkdocs_template: Optional[str] = None
) -> Dict[str, str]:
    config = ConfigParser()
    config.read(settings_path)
    if not mkdocs_template:
        mkdocs_template = _mkdocs_template
    keys = [s[1:-1] for s in re.findall("\{.*?\}", _mkdocs_template)]
    kwargs = {k: config["DEFAULT"][k] for k in keys}
    return kwargs

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    kwargs = _get_kwargs_from_settings(settings_path)

    actual = _mkdocs_template.format(**kwargs)

kwargs

{'title': 'nbdev_mkdocs',
 'doc_host': 'https://airtai.github.io',
 'doc_baseurl': '/nbdev_mkdocs',
 'author': 'airt',
 'description': 'Extension to nbdev for usage of Material for Mkdocs instead of Quarto',
 'repo': 'nbdev_mkdocs',
 'git_url': 'https://github.com/airtai/nbdev_mkdocs',
 'copyright': '2022 onwards, airt'}

In [ ]:
# | export


def _create_mkdocs_yaml(root_path: str):
    try:
        # create mkdocs folder if necessary
        mkdocs_path = Path(root_path) / "mkdocs" / "mkdocs.yml"
        mkdocs_path.parent.mkdir(exist_ok=True)
        # mkdocs.yml already exists, just return
        if mkdocs_path.exists():
            typer.secho(
                f"Path '{mkdocs_path.resolve()}' exists, skipping generation of it."
            )
            return

        # get default values from settings.ini
        settings_path = Path(root_path) / "settings.ini"
        kwargs = _get_kwargs_from_settings(settings_path)
        mkdocs_yaml_str = _mkdocs_template.format(**kwargs)
        with open(mkdocs_path, "w") as f:
            f.write(mkdocs_yaml_str)
            typer.secho(f"File '{mkdocs_path.resolve()}' generated.")
            return
    except Exception as e:
        typer.secho(
            f"Unexpected Error while creating '{mkdocs_path.resolve()}': {e}",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=3)

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    _create_mkdocs_yaml(d)

    with open(Path(d) / "mkdocs/mkdocs.yml") as f:
        y = yaml.safe_load(f)

y

File '/tmp/tmp7grh4x91/mkdocs/mkdocs.yml' generated.


{'site_name': 'nbdev_mkdocs',
 'site_url': 'https://airtai.github.io/nbdev_mkdocs',
 'site_author': 'airt',
 'site_description': 'Extension to nbdev for usage of Material for Mkdocs instead of Quarto',
 'repo_name': 'nbdev_mkdocs',
 'repo_url': 'https://github.com/airtai/nbdev_mkdocs',
 'edit_uri': '',
 'copyright': '2022 onwards, airt',
 'docs_dir': 'docs',
 'site_dir': 'site',
 'plugins': [{'literate-nav': {'nav_file': 'SUMMARY.md'}},
  'search',
  {'mkdocstrings': {'handlers': {'python': {'import': ['https://docs.python.org/3/objects.inv'],
      'options': {'heading_level': 2,
       'show_category_heading': True,
       'show_root_heading': True,
       'show_root_toc_entry': True,
       'show_signature_annotations': True,
       'show_if_no_docstring': True}}}}}],
 'markdown_extensions': [{'pymdownx.arithmatex': {'generic': True}},
  'pymdownx.inlinehilite',
  'pymdownx.details',
  'pymdownx.emoji',
  'pymdownx.magiclink',
  'pymdownx.superfences',
  'pymdownx.tasklist',
  {'pym

### Bringing it all together

In [ ]:
# | export


def new(root_path: str):
    """Initialize mkdocs project files

    Creates **mkdocs** directory in the **root_path** directory and populates
    it with initial values. You should edit mkdocs.yml file to customize it if
    needed.

    Params:
        root_path: path under which mkdocs directory will be created
    """
    _add_requirements_to_settings(root_path)
    _create_mkdocs_dir(root_path)
    _create_mkdocs_yaml(root_path)

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    new(d)

    mkdocs_path = Path(d) / "mkdocs"
    assert settings_path.exists()
    assert mkdocs_path.exists()
    assert (mkdocs_path / "mkdocs.yml").exists()
    assert (mkdocs_path / "overrides" / "main.html").exists()
    assert (mkdocs_path / "site").exists()

Requirements already added to '/tmp/tmpv63i_scp/settings.ini'.
Directory /tmp/tmpv63i_scp/mkdocs created.
File '/tmp/tmpv63i_scp/mkdocs/mkdocs.yml' generated.


## Build

In [ ]:
# | export


def get_submodules(package_name: str) -> List[str]:
    # nosemgrep: python.lang.security.audit.non-literal-import.non-literal-import
    m = importlib.import_module(package_name)
    submodules = [
        info.name
        for info in pkgutil.walk_packages(m.__path__, prefix=f"{package_name}.")
    ]
    submodules = [
        x
        for x in submodules
        if not any([name.startswith("_") for name in x.split(".")])
    ]
    return submodules

In [ ]:
submodules = get_submodules("mkdocs")
submodules

['mkdocs.commands',
 'mkdocs.commands.babel',
 'mkdocs.commands.build',
 'mkdocs.commands.gh_deploy',
 'mkdocs.commands.new',
 'mkdocs.commands.serve',
 'mkdocs.commands.setup',
 'mkdocs.config',
 'mkdocs.config.base',
 'mkdocs.config.config_options',
 'mkdocs.config.defaults',
 'mkdocs.contrib',
 'mkdocs.contrib.search',
 'mkdocs.contrib.search.search_index',
 'mkdocs.exceptions',
 'mkdocs.livereload',
 'mkdocs.localization',
 'mkdocs.plugins',
 'mkdocs.structure',
 'mkdocs.structure.files',
 'mkdocs.structure.nav',
 'mkdocs.structure.pages',
 'mkdocs.structure.toc',
 'mkdocs.tests',
 'mkdocs.tests.babel_cmd_tests',
 'mkdocs.tests.base',
 'mkdocs.tests.build_tests',
 'mkdocs.tests.cli_tests',
 'mkdocs.tests.config',
 'mkdocs.tests.config.base_tests',
 'mkdocs.tests.config.config_options_tests',
 'mkdocs.tests.config.config_tests',
 'mkdocs.tests.gh_deploy_tests',
 'mkdocs.tests.integration',
 'mkdocs.tests.livereload_tests',
 'mkdocs.tests.localization_tests',
 'mkdocs.tests.new_tests

In [ ]:
# | export


def generate_api_doc_for_submodule(root_path: str, submodule: str) -> str:
    subpath = "API/" + submodule.replace(".", "/") + ".md"
    path = Path(root_path) / "mkdocs" / "docs" / subpath
    path.parent.mkdir(exist_ok=True, parents=True)
    with open(path, "w") as f:
        f.write(f"::: {submodule}")
    subnames = submodule.split(".")
    if len(subnames) > 2:
        return " " * 4 * (len(subnames) - 2) + f"- [{subnames[-1]}]({subpath})"
    else:
        return f"- [{submodule}]({subpath})"


def generate_api_docs_for_module(root_path: str, module_name: str) -> str:
    submodules = get_submodules(module_name)
    shutil.rmtree(Path(root_path) / "mkdocs" / "docs" / "API", ignore_errors=True)
    submodule_summary = "\n".join(
        [
            generate_api_doc_for_submodule(root_path=root_path, submodule=x)
            for x in submodules
        ]
    )
    return "- API\n" + textwrap.indent(submodule_summary, prefix=" " * 4)

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    new(d)

    api_summary = generate_api_docs_for_module(d, "mkdocs")
    print(api_summary)

    # make sure all paths exist
    paths = re.findall("\(.*?\)", api_summary)
    paths = [Path(d) / "mkdocs/docs" / x[1:-1] for x in paths]
    for path in paths:
        assert path.exists(), path

Requirements already added to '/tmp/tmp4c663e41/settings.ini'.
Directory /tmp/tmp4c663e41/mkdocs created.
File '/tmp/tmp4c663e41/mkdocs/mkdocs.yml' generated.
- API
    - [mkdocs.commands](API/mkdocs/commands.md)
        - [babel](API/mkdocs/commands/babel.md)
        - [build](API/mkdocs/commands/build.md)
        - [gh_deploy](API/mkdocs/commands/gh_deploy.md)
        - [new](API/mkdocs/commands/new.md)
        - [serve](API/mkdocs/commands/serve.md)
        - [setup](API/mkdocs/commands/setup.md)
    - [mkdocs.config](API/mkdocs/config.md)
        - [base](API/mkdocs/config/base.md)
        - [config_options](API/mkdocs/config/config_options.md)
        - [defaults](API/mkdocs/config/defaults.md)
    - [mkdocs.contrib](API/mkdocs/contrib.md)
        - [search](API/mkdocs/contrib/search.md)
            - [search_index](API/mkdocs/contrib/search/search_index.md)
    - [mkdocs.exceptions](API/mkdocs/exceptions.md)
    - [mkdocs.livereload](API/mkdocs/livereload.md)
    - [mkdocs.locali

In [ ]:
# | export


def build_summary(
    root_path: str,
    module: str,
):
    # create docs_path if needed
    docs_path = Path(root_path) / "mkdocs" / "docs"
    docs_path.mkdir(exist_ok=True)

    # copy README.md as index.md
    shutil.copy(Path(root_path) / "README.md", docs_path / "index.md")

    api_summary = generate_api_docs_for_module(root_path, module)

    summary = "- [Home](index.md)\n"
    summary = summary + api_summary

    with open(docs_path / "SUMMARY.md", mode="w") as f:
        f.write(summary)

    return summary

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    for fname in ["settings.ini", "README.md"]:
        shutil.copyfile(Path("..") / fname, Path(d) / fname)

    new(d)

    build_summary(d, "mkdocs")

    #     Path(d) / (d, "mkdocs")
    #     api_summary = generate_api_docs_for_module(d, "mkdocs")
    #     print(api_summary)

    #     # make sure all paths exist
    #     paths = re.findall("\(.*?\)", api_summary)
    #     paths = [Path(d) / "mkdocs/docs" / x[1:-1] for x in paths]
    #     for path in paths:
    #         assert path.exists(), path

    with open(Path(d) / "mkdocs/docs/SUMMARY.md") as f:
        summary = f.read()

    print(summary)

Requirements already added to '/tmp/tmploshqpvs/settings.ini'.
Directory /tmp/tmploshqpvs/mkdocs created.
File '/tmp/tmploshqpvs/mkdocs/mkdocs.yml' generated.
- [Home](index.md)
- API
    - [mkdocs.commands](API/mkdocs/commands.md)
        - [babel](API/mkdocs/commands/babel.md)
        - [build](API/mkdocs/commands/build.md)
        - [gh_deploy](API/mkdocs/commands/gh_deploy.md)
        - [new](API/mkdocs/commands/new.md)
        - [serve](API/mkdocs/commands/serve.md)
        - [setup](API/mkdocs/commands/setup.md)
    - [mkdocs.config](API/mkdocs/config.md)
        - [base](API/mkdocs/config/base.md)
        - [config_options](API/mkdocs/config/config_options.md)
        - [defaults](API/mkdocs/config/defaults.md)
    - [mkdocs.contrib](API/mkdocs/contrib.md)
        - [search](API/mkdocs/contrib/search.md)
            - [search_index](API/mkdocs/contrib/search/search_index.md)
    - [mkdocs.exceptions](API/mkdocs/exceptions.md)
    - [mkdocs.livereload](API/mkdocs/livereload.md)
 

In [ ]:
# | export


def prepare(root_path: str):
    """Prepares mkdocs for serving

    Params:
        root_path: path under which mkdocs directory will be created
    """
    # get lib name from settings.ini
    settings_path = Path(root_path) / "settings.ini"
    config = ConfigParser()
    config.read(settings_path)
    lib_name = config["DEFAULT"]["lib_name"]

    build_summary(root_path, lib_name)

    cmd = f"mkdocs build -f {root_path}/mkdocs/mkdocs.yml"
    
    # nosemgrep: python.lang.security.audit.subprocess-shell-true.subprocess-shell-true
    sp = subprocess.run( # nosec: B602:subprocess_popen_with_shell_equals_true
        cmd,
        shell=True,
        #         check=True,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
    )
    print(sp.stdout)
    if sp.returncode != 0:
        typer.secho(
            f"Command '{cmd}' failed!",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(5)

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    for fname in ["settings.ini", "README.md"]:
        shutil.copyfile(Path("..") / fname, Path(d) / fname)

    new(d)

    prepare(d)

    assert (Path(d) / "mkdocs" / "docs" / "API").exists
    assert (Path(d) / "mkdocs" / "docs" / "SUMMARY.md").exists
    assert (Path(d) / "mkdocs" / "docs" / "index.md").exists
#     !ls {d}/mkdocs/docs

Requirements already added to '/tmp/tmp5zdwgnad/settings.ini'.
Directory /tmp/tmp5zdwgnad/mkdocs created.
File '/tmp/tmp5zdwgnad/mkdocs/mkdocs.yml' generated.
/root/.local/lib/python3.8/site-packages/mkdocstrings/handlers/python/__init__.py:13: UserWarning: The 'python-legacy' extra of mkdocstrings will become mandatory in the next release. We have no way to detect if you already specify it, so if you do, please ignore this warning. You can globally disable it with the PYTHONWARNINGS environment variable: PYTHONWARNINGS=ignore::UserWarning:mkdocstrings.handlers.python
  warnings.warn(
INFO     -  Cleaning site directory
INFO     -  Building documentation to directory: /tmp/tmp5zdwgnad/mkdocs/site
WARNING  -  mkdocstrings: nbdev_mkdocs.old.A.f: Failed to get 'name: description' pair from ''
INFO     -  Documentation built in 0.62 seconds



## Preview

In [ ]:
# | export

import shlex


def preview(root_path: str, port: Optional[int] = None):
    """Previes mkdocs documentation

    Params:
        root_path: path under which mkdocs directory will be created
        port: port to use
    """
    cmd = f"mkdocs serve -f {root_path}/mkdocs/mkdocs.yml -a 0.0.0.0"
    if port:
        cmd = cmd + f":{port}"

    with subprocess.Popen( #nosec B603:subprocess_without_shell_equals_true
        shlex.split(cmd),
        stdout=subprocess.PIPE,
        bufsize=1,
        text=True,
        universal_newlines=True,
    ) as p:
        for line in p.stdout:  # type: ignore
            print(line, end="")

    if p.returncode != 0:
        typer.secho(
            f"Command '{cmd}' failed!",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(6)

In [ ]:
# with TemporaryDirectory() as d:
#     settings_path = Path(d) / "settings.ini"
#     for fname in ["settings.ini", "README.md"]:
#         shutil.copyfile(Path("..") / fname, Path(d) / fname)

#     new(d)

#     prepare(d)

#     preview(d, port=4000)

#     assert (Path(d) / "mkdocs" / "docs" / "API").exists
#     assert (Path(d) / "mkdocs" / "docs" / "SUMMARY.md").exists
#     assert (Path(d) / "mkdocs" / "docs" / "index.md").exists
# #     !ls {d}/mkdocs/docs